In [1]:
# import libraries

import itertools
import glob
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os


# type in the information in this cell

In [2]:
# Input information in this cells

## Set number
SetNumber = 'set2'

## path
os.chdir(r'O:\ImStor\sorger\data\Cytell\Kyun\cycIF_analysis_python_v2\set2_20170322_4CL_5h5d_pERK_singleCell')


## cycle number - how many rounds in cycIF?

TotalCycleNumber = 2     #cycIF cycle number


## timepoints by Row and Column

Timepoint1 = '5h' #Row BCD, Col2-4
Timepoint2 = '1d' #Row BCD, Col5-7
Timepoint3 = '2d' #Row BCD, Col8-10

Timepoint4 = '3d' #Row EFG, Col2-4
Timepoint5 = '4d' #Row EFG, Col5-7
Timepoint6 = '5d' #Row EFG, Col8-10


## Fill in the antibody information

Channel = pd.Series(['DAPI-0001', 'FITC-0001', 'Cy3-0001', 'Cy5-0001',
                     'DAPI-0002', 'FITC-0002', 'Cy3-0002', 'Cy5-0002',
                     'DAPI-0003', 'FITC-0003', 'Cy3-0003', 'Cy5-0003',
                     'DAPI-0004', 'FITC-0004', 'Cy3-0004', 'Cy5-0004',
                     ])

Staining_BCDEFG = pd.Series(['DAPI1', 'FITC-0001', 'Cy3-0001', 'pErk',
                     'DAPI2', 'Ki67', 'pRB', 'pS6-235',
                     'DAPI-0003', 'FITC-0003', 'Cy3-0003', 'Cy5-0003',
                     'DAPI-0004', 'FITC-0004', 'Cy3-0004', 'Cy5-0004',
                     ])


# doublecheck from Matlab
#     % Column 1-10: DAPI Cycle0 (Nuc 1-5, Cyto 6-10)
#     % Column 11-20: DAPI Cycle1
#     % Column 21-30: Cy3 Cycle0 
#     % Column 31-40: Cy3 Cycle1 :pRB

#     % Column 41-50: Cy5 Cycle0 :pErk
#     % Column 51-60: Cy5 Cycle1 :pS6-235

#     % Column 61-70: FITC Cycle0 
#     % Column 71-80: FITC Cycle1 :Ki67

# run

In [3]:
## This is for metadata_set*_Well.csv

# Create lists with the different factors.
rows = ['B', 'C', 'D', 'E', 'F', 'G']
columns = range(2, 10 + 1)

# Generate a dataframe with all combinations of all factors (Cartesian product).
design = pd.DataFrame(list(itertools.product(
    rows, columns
)))
design.columns = ['Row', 'Column']

# Add some helpful columns that are made up by combining other factors.
design['Well'] = (
    design['Row']
    + design['Column'].astype(str).str.pad(2, fillchar='0')
)

# BCD rows and EFG rows
design.loc[design['Row'].isin(['B', 'C', 'D']), 'Top/Bottom'] = 'BCD'
design.loc[design['Row'].isin(['E', 'F', 'G']), 'Top/Bottom'] = 'EFG'

# Fill in the Treatment by applying the below "matrix" over patterns on Row and
# Column.
design['Treatment'] = None
trt_matrix = [
    ['DMSO', 'Tram', 'Fulv'],
    ['BYL0.03', 'BYL0.3', 'BYL3'],
    ['F+BYL0.03', 'F+BYL0.3', 'F+BYL3']
]

tm_cols = [[2,5,8], [3,6,9], [4,7,10]]
tm_rows = [['B', 'E'], ['C', 'F'], ['D', 'G']]

for c, columns in enumerate(tm_cols):
    for r, rows in enumerate(tm_rows):
        idx = (design['Column'].isin(columns)) & (design['Row'].isin(rows))
        design.loc[idx, 'Treatment'] = trt_matrix[c][r]
        
# Fill in the time points
design.loc[ (  (design['Row'].isin(['B', 'C', 'D'])) & (design['Column'].isin([2, 3, 4]))  ), 'Time'] = Timepoint1
design.loc[ (  (design['Row'].isin(['B', 'C', 'D'])) & (design['Column'].isin([5, 6, 7]))  ), 'Time'] = Timepoint2
design.loc[ (  (design['Row'].isin(['B', 'C', 'D'])) & (design['Column'].isin([8, 9, 10]))  ), 'Time'] = Timepoint3

design.loc[ (  (design['Row'].isin(['E', 'F', 'G'])) & (design['Column'].isin([2, 3, 4]))  ), 'Time'] = Timepoint4
design.loc[ (  (design['Row'].isin(['E', 'F', 'G'])) & (design['Column'].isin([5, 6, 7]))  ), 'Time'] = Timepoint5
design.loc[ (  (design['Row'].isin(['E', 'F', 'G'])) & (design['Column'].isin([8, 9, 10]))  ), 'Time'] = Timepoint6

# save metadata file
fileName1 = 'metadata_' + SetNumber + '_Well.csv'
design.to_csv(fileName1)

In [4]:
## This is for metadata_set*_Staining.csv

design_Staining_BCDEFG = pd.DataFrame({'Channel':Channel[:4*TotalCycleNumber],
                              'Staining': Staining_BCDEFG[:4*TotalCycleNumber]
                               })

design_Staining_BCDEFG['Top/Bottom'] = 'BCDEFG'


Staining_all = design_Staining_BCDEFG

# Save metadata file for staining
fileName2 = 'metadata_' + SetNumber + '_Staining.csv'
Staining_all.to_csv(fileName2)